### About this result

Test the pretrained model (trained with 8 particles per event) with different number of particles.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import g_main

sns.set_theme()

In [ ]:
# Range of different number of particles.
diff_ptcs_range = range(2, 16 + 1)

# The csv files that stored the prediction information.
pred_dir = f"{g_main.json_config['predictions_dir']}/ideal_model"
num_bin_data = 500 # Number of data in a bin.
num_rnd_seed = 3 # Number of random seeds that has been predicted.
csv_best = f"{pred_dir}/best_classical-{num_bin_data}_{num_rnd_seed - 1}.csv"
csv_classical = f"{pred_dir}/classical-{num_bin_data}_{num_rnd_seed - 1}.csv"
csv_quantum = f"{pred_dir}/quantum-{num_bin_data}_{num_rnd_seed - 1}.csv"

# Create pandas dataframes.
df_best = pd.read_csv(csv_best)
df_classical = pd.read_csv(csv_classical)
df_quantum = pd.read_csv(csv_quantum)
df = pd.concat((df_best, df_classical, df_quantum))

# Fill NaN or None with value 0.
df = df.fillna(0)

# Retain columns we need.
retain_info = [
    "model_name", "abbrev", "data_mode", "num_pt_ptcs", "test_acc_epoch",
    "rnd_seed", "gnn_out", "gnn_hidden", "gnn_layers",
    "gnn_nn_qubits", "gnn_reupload"
]
df = df[retain_info]

# Create new columns for seaborn `hue`.
df.loc[df["gnn_out"].astype(int) == 1024, "M"] = "B"     # Best classical.
df.loc[df["model_name"] == "ClassicalMPGNN", "M"] = "C"  # Classical.
df.loc[df["model_name"] == "QuantumRotQCGNN", "M"] = "Q" # Quantum.

# Key "data_mode" in df cooresponds to training or testing data.
df["hue"] = df["M"] + "_" + df["data_mode"]
print(f"Hue for seaborn: {df['hue'].unique()}")
df

In [ ]:
# Figure setup.
abbrev_list = ["BB-QCD", "TT-QCD"]
dim_list = [3, 6, 9]
num_row = len(abbrev_list)
num_col = len(dim_list)
width, height = 5 * num_col, 5 * num_row
fig, ax = plt.subplots(num_row, num_col, figsize=(width, height))

for i, abbrev in enumerate(abbrev_list):
    for j, dim in enumerate(dim_list):
        
        # Indices with corresponding dimension.
        is_dim = (
            # Best classical and classical with dimension `dim`.
            (df["M"] ==  "B") | (df["gnn_out"].astype(int) == dim) | \
            # Quantum with dimension `dim`.
            (df["gnn_nn_qubits"].astype(int) == dim)
        )

        # Indices with corresponding abbrev (channels of jet discrimination).
        is_abbrev = (df["abbrev"] == abbrev)

        # Set y-axis limits.
        ax[i][j].set_ylim(0.4, 0.9)

        # Line plot.
        sns.lineplot(
            data=df[is_dim & is_abbrev], ax=ax[i][j],
            x="num_pt_ptcs", y="test_acc_epoch", hue="hue",
            errorbar="se", marker="o"
        )

        # # Bar plot.
        # sns.barplot(
        #     data=df[is_dim & is_abbrev], ax=ax[1][i],
        #     x="num_pt_ptcs", y="test_acc_epoch", hue="hue",
        #     errorbar=None
        # )

plt.tight_layout()